In [1]:
import cv2
import numpy as np
import mediapipe as mp
import os
import random
from mtcnn import MTCNN
from PIL import Image, ImageEnhance

In [2]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [3]:
face_dir = "faces/"
detector = MTCNN()

used_ids = []

In [4]:
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

def adjust_brightness_contrast(image, brightness=0, contrast=0):
    image = Image.fromarray(image)
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(1 + brightness / 100.0)
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1 + contrast / 100.0)
    image = np.array(image)
    return image

def augment_image(img):
    angle = random.randint(-10, 10)
    rows, cols, _ = img.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    img = cv2.warpAffine(img, M, (cols, rows))
    if random.random() > 0.5:
        img = cv2.flip(img, 1)
    brightness = random.randint(10, 40)
    img = adjust_brightness_contrast(img, brightness=brightness)
    return img

def detect_face_pose(face_landmarks, img_w, img_h):
    face_3d = []
    face_2d = []

    for idx, lm in enumerate(face_landmarks.landmark):
        if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
            if idx == 1:
                nose_2d = (lm.x * img_w, lm.y * img_h)
                nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
            x, y = int(lm.x * img_w), int(lm.y * img_h)
            face_2d.append([x, y])
            face_3d.append([x, y, lm.z])

    face_2d = np.array(face_2d, dtype=np.float64)
    face_3d = np.array(face_3d, dtype=np.float64)

    focal_length = 1 * img_w
    cam_matrix = np.array([[focal_length, 0, img_h / 2],
                           [0, focal_length, img_w / 2],
                           [0, 0, 1]])
    dist_matrix = np.zeros((4, 1), dtype=np.float64)

    success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
    rmat, jac = cv2.Rodrigues(rot_vec)
    angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

    x = angles[0] * 360
    y = angles[1] * 360
    z = angles[2] * 360

    if y < -10:
        return "Looking Left", x, y, z
    elif y > 10:
        return "Looking Right", x, y, z
    elif x < -10    :
        return "Looking Down", x, y, z
    elif x > 15:
        return "Looking Up", x, y, z
    else:
        return "Forward", x, y, z
    
def main():
    create_folder(face_dir)
    name = input("Enter name: ")

    while True:    
        try:
            face_id = int(input("Enter id for face: "))
            
            # Check if the name exists in used_ids, if not create a new entry 
            if face_id not in used_ids:
                used_ids.append(face_id)
            else:
                print("ID already exists. Please enter a different ID.")
                continue
            
            face_id = int(face_id)
            face_folder = face_dir + name + "_" + str(face_id) + "/"
            
            create_folder(face_folder)
            break
        
        except ValueError:
            print("Invalid input. ID must be an integer.")
            continue
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            continue

    while True:
        total_images = input("Enter the number of images you want to have: ")
        try:
            total_images = int(total_images)
            break
        except:
            print("The number of images should be an integer.")
            continue

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FPS, 60)

    poses = ["Forward", "Looking Left", "Looking Right", "Looking Up", "Looking Down"]
    pose_counts = {pose: 0 for pose in poses}
    current_pose_index = 0

    while current_pose_index < len(poses):
        current_pose = poses[current_pose_index]
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = detector.detect_faces(img_rgb)

        if len(faces) == 1:
            face = faces[0]
            x, y, w, h = face["box"]
            face_img = img_rgb[y:y+h, x:x+w]
            face_img = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)
            img_h, img_w, img_c = img.shape

            results = face_mesh.process(face_img)
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    pose, x_angle, y_angle, z_angle = detect_face_pose(face_landmarks, img_w, img_h)

                    if pose == current_pose:
                        img_path = face_folder + f"{name}_{pose}_{pose_counts[pose]+1}.jpg"
                        cv2.imwrite(img_path, face_img)
                        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 3)
                        pose_counts[pose] += 1

                        if augment_status:
                            for i in range(3):
                                augmented_img = augment_image(face_img)
                                augmented_img_path = face_folder + f"{name}_{pose}_{pose_counts[pose]+1}.jpg"
                                cv2.imwrite(augmented_img_path, augmented_img)
                                #pose_counts[pose] += 1

                    cv2.putText(img, f"Current pose: {current_pose}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(img, f"Captured: {pose_counts[current_pose]}/{total_images}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Saving", img)
        if cv2.waitKey(1) & 0xff == ord("q"):
            break
        if pose_counts[current_pose] >= total_images:
            current_pose_index += 1

    cap.release()
    cv2.destroyAllWindows()
    print("Face has been created.")

In [5]:
main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step


c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━